In [10]:
import pandas as pd 
import numpy as np 

# Import TF Functions
import sys 
sys.path.append(r'C:\Users\kurt-\Documents\GitHub\GNN_Soil_Dynamics\TF Functions')
from TTF import TTF

In [11]:
# Read all files with csv
import os
import re
import matplotlib.pyplot as plt
path = r"FLAC\FLAC2D\1D Multiple columns\Results"

files = [ f for f in os.listdir(path) if f.endswith('.csv')]

# Extract the index saved with the files
indices = []
for file in files:
    match = re.search(r'Data_(\d+\.\d+)_\.csv', file)
    if match:
        index = float(match.group(1))
        indices.append(index)

# Print the extracted indices
print(indices)

# Reorder the Vs arrays
Vs_arrays = np.genfromtxt(r"FLAC\FLAC2D\1D Multiple columns\model_arrays.csv", delimiter=",", skip_header=1)
# Convert indices to integer if they are not already
indices = [int(i) for i in indices]

# Reorder Vs_arrays based on indices
Vs_arrays = Vs_arrays[indices, :]

# Print the reordered Vs_arrays to verify
print(Vs_arrays)


[0.0, 1.0, 10.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 11.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 12.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 13.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 14.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 15.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 159.0, 16.0, 160.0, 161.0, 162.0, 163.0, 164.0, 165.0, 166.0, 167.0, 168.0, 169.0, 17.0, 170.0, 171.0, 172.0, 173.0, 174.0, 175.0, 176.0, 177.0, 178.0, 179.0, 18.0, 180.0, 181.0, 182.0, 183.0, 184.0, 185.0, 186.0, 187.0, 188.0, 189.0, 19.0, 190.0, 191.0, 192.0, 193.0, 194.0, 195.0, 196.0, 197.0, 198.0, 199.0, 2.0, 20.0, 200.0, 201.0, 202.0, 203.0, 204.0, 205.0, 206.0, 207.0, 208.0, 209.0, 21.0, 210.0, 211.0, 212.0, 213.0, 214.0, 215.0, 216.0, 217.0, 218.0, 219.0, 22.0, 220.0, 221.0, 222.0, 223.0, 224.0, 225.0, 226.0, 227.0, 228.0, 229

In [12]:
# Create a class that reads the csv and appends the TTF data
class TTF_Reader:
    def __init__(self, path):
        self.path = path
        self.files = [ f for f in os.listdir(path) if f.endswith('.csv')]
        self.data = []
        
    def read(self):
        prev_freq = None
        save = 0
        for file in self.files:
            data = pd.read_csv(os.path.join(self.path, file))
            freq, TTF_val = TTF(data.iloc[:, 2].values, data.iloc[:, 1].values, data.iloc[0, 0])
            # Check if the frequency is the same
            if prev_freq is not None:
                if np.allclose(prev_freq, freq) == False:
                    print(f"Frequency is not the same for file {file}")
                    break
                else:
                    prev_freq = freq
                    
            if save == 0:
                np.savetxt(f"TTF_freq.csv", freq, delimiter=',')
                save = 1
            # Just append TTF
            self.data.append(TTF_val)
            print(f"File {file} has been read")
            
        return self.data

In [13]:
A = TTF_Reader(path)
A.read()


File Data_0.000_.csv has been read
File Data_1.000_.csv has been read
File Data_10.000_.csv has been read
File Data_100.000_.csv has been read
File Data_101.000_.csv has been read
File Data_102.000_.csv has been read
File Data_103.000_.csv has been read
File Data_104.000_.csv has been read
File Data_105.000_.csv has been read
File Data_106.000_.csv has been read
File Data_107.000_.csv has been read
File Data_108.000_.csv has been read
File Data_109.000_.csv has been read
File Data_11.000_.csv has been read
File Data_110.000_.csv has been read
File Data_111.000_.csv has been read
File Data_112.000_.csv has been read
File Data_113.000_.csv has been read
File Data_114.000_.csv has been read
File Data_115.000_.csv has been read
File Data_116.000_.csv has been read
File Data_117.000_.csv has been read
File Data_118.000_.csv has been read
File Data_119.000_.csv has been read
File Data_12.000_.csv has been read
File Data_120.000_.csv has been read
File Data_121.000_.csv has been read
File Dat

[array([ 1.02760104,  1.02760104,  1.02821657,  1.02892606,  1.02965874,
         1.03040893,  1.03117755,  1.03196849,  1.03278031,  1.03361294,
         1.03446716,  1.03534419,  1.03624423,  1.03716771,  1.03811545,
         1.0390883 ,  1.04008678,  1.0411115 ,  1.04216334,  1.04324311,
         1.04435152,  1.04548928,  1.04665738,  1.04785667,  1.04908799,
         1.05035219,  1.05165037,  1.05298346,  1.05435241,  1.05575826,
         1.05720217,  1.05868525,  1.06020859,  1.06177338,  1.06338093,
         1.06503247,  1.06672929,  1.06847274,  1.07026432,  1.07210544,
         1.07399758,  1.07594229,  1.07794128,  1.07999619,  1.08210872,
         1.08428066,  1.08651396,  1.08881052,  1.09117231,  1.09360145,
         1.09610016,  1.09867062,  1.10131516,  1.10403623,  1.10683639,
         1.10971821,  1.11268441,  1.11573784,  1.11888148,  1.12211839,
         1.12545173,  1.12888486,  1.13242131,  1.13606468,  1.1398187 ,
         1.14368735,  1.14767477,  1.15178523,  1.1

In [14]:
# Save the data to a pt file
import pickle

with open(os.path.join(path,'TTF_data.pt'), 'wb') as f:
    pickle.dump(A.data, f)
    print('Data has been saved')


Data has been saved


In [15]:

# Save the Vs values
with open(os.path.join(path,'Vs_values.pt'), 'wb') as f:
    pickle.dump(Vs_arrays, f)
    print('Vs values have been saved')

Vs values have been saved
